# do average voting using multiple Light Gradient Boosting Machines

## import libraries

In [1]:
import joblib
import multiprocessing
import os
import lightgbm as lgb
import numpy as np
import pandas as pd

from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

## change working directory

In [2]:
os.chdir('..')

## define paths

In [2]:
model_dir_path = 'models/lgbm/'
output_dir_path = 'output/lgbm/'

os.makedirs(model_dir_path, exist_ok=True)
os.makedirs(output_dir_path, exist_ok=True)

train_transaction_data_path = 'data/train_transaction.csv'
train_identity_data_path = 'data/train_identity.csv'
test_transaction_data_path = 'data/test_transaction.csv'
test_identity_data_path = 'data/test_identity.csv'

## define utility function to reduce memory usage

In [3]:
def reduce_mem_usage(df, verbose=True):
    """
    Reduce dataframe size

    params:
    - df: dataframe to reduce the size of

    return:
    - dataframe of reduced size
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64', 'float128']
    start_mem = df.memory_usage().sum() / 1024**2    

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float64).min and c_max < np.finfo(np.float64).max:
                    df[col] = df[col].astype(np.float64)
                elif c_min > np.finfo(np.float128).min and c_max < np.finfo(np.float128).max:
                    df[col] = df[col].astype(np.float128)
                    
    end_mem = df.memory_usage().sum() / 1024**2

    if verbose: 
        print(
            'Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(
            end_mem, 100 * (start_mem - end_mem) / start_mem
        ))

    return df

## list down useless features (known from feature selection)

In [4]:
useless_features = [
    'TransactionID',  # not really a feature
    'dist2',  # transaction features
    'C3',  # C features
    'D6', 'D7', 'D8', 'D9', 'D12', 'D13', 'D14',  # D features
    'M1',  # M features
    'id_07', 'id_08', 'id_18', 'id_21', 'id_22', 'id_23',  # id features
    'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_35',  # id features
    'V6', 'V8', 'V9', 'V10', 'V11', 'V14', 'V15', 'V16',  # V features
    'V18', 'V21', 'V22', 'V27', 'V28', 'V31', 'V32',  # V features
    'V41', 'V42', 'V46', 'V50', 'V51', 'V59', 'V65',  # V features
    'V68', 'V71', 'V72', 'V79', 'V80', 'V84', 'V85',  # V features
    'V88', 'V89', 'V92', 'V93', 'V95', 'V98', 'V101',  # V features
    'V104', 'V106', 'V107', 'V108', 'V109', 'V110',  # V features
    'V111', 'V112', 'V113', 'V114', 'V116', 'V117',  # V features
    'V118', 'V119', 'V120', 'V121', 'V122', 'V123',  # V features 
    'V125', 'V138', 'V141', 'V142', 'V144', 'V146',  # V features 
    'V147', 'V148', 'V151', 'V153', 'V154', 'V155',  # V features 
    'V157', 'V158', 'V159', 'V161', 'V163', 'V164',  # V features 
    'V166', 'V172', 'V173', 'V174', 'V175', 'V176',  # V features 
    'V177', 'V178', 'V179', 'V180', 'V181', 'V182',  # V features  
    'V183', 'V184', 'V185', 'V186', 'V190', 'V191',  # V features  
    'V192', 'V193', 'V194', 'V195', 'V196', 'V197',  # V features  
    'V198', 'V199', 'V214', 'V216', 'V220', 'V225',  # V features 
    'V226', 'V227', 'V230', 'V233', 'V235', 'V236',  # V features  
    'V237', 'V238', 'V239', 'V240', 'V241', 'V242',  # V features 
    'V244', 'V246', 'V247', 'V248', 'V249', 'V250',  # V features 
    'V252', 'V254', 'V255', 'V269', 'V276', 'V297',  # V features 
    'V300', 'V302', 'V304', 'V305', 'V325', 'V327',  # V features  
    'V328', 'V329', 'V330', 'V334', 'V335', 'V336',  # V features 
    'V337', 'V338', 'V339',  # V features 
]


## define function to disregard OS versions

In [5]:
def ignore_os_version(df, verbose: bool=True):
    """
    params:
    - df (DataFrame): has id_30 as one of its columns
    - verbose (bool): prints information if True

    return: dataframe, after os versions have been ignored
    """
    os_list = [
        'Android',
        'iOS',
        'Mac OS X',
        'Windows',
    ]

    for index, operating_system in df.id_30.iteritems():
        new_os = 'other'

        if isinstance(operating_system, str):
            for known_os in os_list:
                if known_os in operating_system:
                    new_os = known_os
                    break

        df.at[index, 'id_30'] = new_os

    if verbose:
        print('operating systems:', df.id_30.unique())

    return df

## define function to disregard browser versions

In [6]:
def ignore_browser_version(df, verbose: bool=True):
    """
    params:
    - df (DataFrame): has id_31 as one of its columns
    - verbose (bool): prints information if True

    return: dataframe, after browser versions have been ignored
    """
    browser_list = [
        'aol',
        'chrome',
        'chromium',
        'comodo',
        'cyberfox',
        'edge',
        'firefox',
        'icedragon',
        'ie',
        'iron',
        'maxthon',
        'opera',
        'palemoon',
        'puffin',
        'safari',
        'samsung',
        'seamonkey',
        'silk',
        'waterfox',
    ]

    for index, browser in df.id_31.iteritems():
        new_browser = 'other'

        if isinstance(browser, str):
            for known_browser in browser_list:
                if known_browser in browser:
                    new_browser = known_browser
                    break

        df.at[index, 'id_31'] = new_browser

    if verbose:
        print('browsers:', df.id_31.unique())

    return df

## define function for preprocessing data

In [7]:
def preprocess(df, verbose: bool=True):
    """
    Does the following preprocessing steps:
    - disregard os versions
    - disregard browser versions
    - drop useless features
    - convert object columns to string columns
    - imputation (for numbers, fill with interquartile mean)
    - do label encoding for non-numeric values
    - reduce memory usage again

    params:   
    - df (DataFrame): dataframe to preprocess (has columns id_30 and id_31)
    - verbose (bool): prints information if True

    return: dataframe, preprocessing is complete
    """
    df = df.drop(useless_features, axis=1)
    df = ignore_os_version(df, verbose)
    df = ignore_browser_version(df, verbose)

    le = LabelEncoder()

    for column in df.columns:
        if df[column].dtype == 'object':
            df[column]= df[column].astype(str)
            df[column] = le.fit_transform(df[column])
        else:
            df[column] = df[column].fillna(df[column].quantile().mean())

    df = reduce_mem_usage(df, verbose)

    return df

## load and preprocess training data

In [8]:
%%time

transaction_dataframe = pd.read_csv(train_transaction_data_path)
transaction_dataframe = reduce_mem_usage(transaction_dataframe)

identity_dataframe = pd.read_csv(train_identity_data_path)
identity_dataframe = reduce_mem_usage(identity_dataframe)

dataframe = transaction_dataframe.merge(identity_dataframe, how='outer')

del transaction_dataframe
del identity_dataframe

print(f'number of rows in training data: {len(dataframe)}')
dataframe = preprocess(dataframe)

dataframe.head()

Mem. usage decreased to 542.35 Mb (69.4% reduction)
Mem. usage decreased to 25.86 Mb (42.7% reduction)
number of rows in training data: 590540
operating systems: ['other' 'Android' 'iOS' 'Mac OS X' 'Windows']
browsers: ['other' 'samsung' 'safari' 'chrome' 'edge' 'firefox' 'ie' 'opera' 'aol'
 'silk' 'waterfox' 'puffin' 'cyberfox' 'palemoon' 'maxthon' 'iron'
 'seamonkey' 'comodo' 'chromium' 'icedragon']
Mem. usage decreased to 357.35 Mb (22.1% reduction)
CPU times: user 1min 16s, sys: 37 s, total: 1min 53s
Wall time: 1min 54s


,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_30,id_31,id_32,id_33,id_34,id_36,id_37,id_38,DeviceType,DeviceInfo
0,0,86400,68.5,4,13926,361.0,150.0,1,142.0,1,...,4,12,24.0,260,4,2,2,2,2,1742
1,0,86401,29.0,4,2755,404.0,150.0,2,102.0,1,...,4,12,24.0,260,4,2,2,2,2,1742
2,0,86469,59.0,4,4663,490.0,150.0,4,166.0,2,...,4,12,24.0,260,4,2,2,2,2,1742
3,0,86499,50.0,4,18132,567.0,150.0,2,117.0,2,...,4,12,24.0,260,4,2,2,2,2,1742
4,0,86506,50.0,1,4497,514.0,150.0,2,102.0,1,...,0,16,32.0,164,3,0,1,1,1,954


## separate data into features and target variable

In [9]:
features_dataframe = dataframe.drop('isFraud', axis=1)
is_fraud_data = dataframe['isFraud']

del dataframe

## define number of base classifiers

In [10]:
num_base_classifiers = 10
offset = 10

## train LightGBM

- using [reference notebook parameters](https://www.kaggle.com/nroman/lgb-single-model-lb-0-9419)

In [11]:
params = {
    'num_leaves': 491,
    'min_child_weight': 0.03454472573214212,
    'feature_fraction': 0.3797454081646243,
    'bagging_fraction': 0.4181193142567742,
    'min_data_in_leaf': 106,
    'objective': 'binary',
    'max_depth': -1,
    'learning_rate': 0.006883242363721497,
    'boosting_type': 'gbdt',
    'bagging_seed': 11,
    'metric': 'auc',
    'verbosity': -1,
    'reg_alpha': 0.3899927210061127,
    'reg_lambda': 0.6485237330340494,
    'random_state': 47,
}

In [16]:
%%time

for index in range(num_base_classifiers):
    classifier_num = index + offset
    print(f'working on base classifier {classifier_num}')

    train_features, val_features, train_target, val_target = train_test_split(
        features_dataframe, 
        is_fraud_data, 
        test_size=0.2,
    )

    train_data = lgb.Dataset(train_features, train_target)
    del train_features
    del train_target

    val_data = lgb.Dataset(val_features, val_target)

    params['random_state'] = classifier_num

    classifier = lgb.train(
        params, 
        train_set=train_data, 
        num_boost_round=10000, 
        valid_sets=[train_data, val_data],
        verbose_eval=1000,
        early_stopping_rounds=500,
    )

    joblib.dump(classifier, model_dir_path + 'lgbm_' + str(classifier_num) + '.joblib')

    prediction = classifier.predict(val_features)
    auc = roc_auc_score(val_target, prediction)
    print(f'base classifier {classifier_num} AUC: {auc}')

del features_dataframe
del is_fraud_data

del params

working on base classifier 10
Training until validation scores don't improve for 500 rounds
[1000]	training's auc: 0.99333	valid_1's auc: 0.96606
[2000]	training's auc: 0.999152	valid_1's auc: 0.973492
[3000]	training's auc: 0.999892	valid_1's auc: 0.975388
[4000]	training's auc: 0.999992	valid_1's auc: 0.976055
[5000]	training's auc: 1	valid_1's auc: 0.976313
Early stopping, best iteration is:
[5110]	training's auc: 1	valid_1's auc: 0.976346
base classifier 10 AUC: 0.9763458399638495
working on base classifier 11
Training until validation scores don't improve for 500 rounds
[1000]	training's auc: 0.993606	valid_1's auc: 0.963698
[2000]	training's auc: 0.999204	valid_1's auc: 0.971077
[3000]	training's auc: 0.999889	valid_1's auc: 0.972926
[4000]	training's auc: 0.999991	valid_1's auc: 0.973595
[5000]	training's auc: 0.999999	valid_1's auc: 0.97369
Early stopping, best iteration is:
[5033]	training's auc: 0.999999	valid_1's auc: 0.9737
base classifier 11 AUC: 0.9737000511033413
working

## load and preprocess test data

In [17]:
%%time

transaction_dataframe = pd.read_csv(test_transaction_data_path)
transaction_dataframe = reduce_mem_usage(transaction_dataframe)

identity_dataframe = pd.read_csv(test_identity_data_path)
identity_dataframe = reduce_mem_usage(identity_dataframe)
identity_dataframe = identity_dataframe.rename(
    columns={
        column: column.replace('-', '_')
        for column in identity_dataframe.columns
    }
)

dataframe = transaction_dataframe.merge(identity_dataframe, how='outer')
transaction_id_data = dataframe['TransactionID']  # need it for output

del transaction_dataframe
del identity_dataframe

print(f'number of rows in test data: {len(dataframe)}')
dataframe = preprocess(dataframe)

dataframe.head()

Mem. usage decreased to 472.59 Mb (68.9% reduction)
Mem. usage decreased to 25.44 Mb (42.7% reduction)
number of rows in test data: 506691
operating systems: ['other' 'Android' 'iOS' 'Windows' 'Mac OS X']
browsers: ['other' 'chrome' 'ie' 'safari' 'edge' 'firefox' 'samsung' 'opera'
 'palemoon']
Mem. usage decreased to 315.73 Mb (21.6% reduction)
CPU times: user 1min 1s, sys: 30.8 s, total: 1min 32s
Wall time: 1min 32s


,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,id_30,id_31,id_32,id_33,id_34,id_36,id_37,id_38,DeviceType,DeviceInfo
0,18403224,31.953125,4,10409,111.0,150.0,4,226.0,2,170.0,...,4,5,24.0,390,2,2,2,2,2,2184
1,18403263,49.000000,4,4272,111.0,150.0,4,226.0,2,299.0,...,4,5,24.0,390,2,2,2,2,2,2184
2,18403310,171.000000,4,4476,574.0,150.0,4,226.0,2,472.0,...,4,5,24.0,390,2,2,2,2,2,2184
3,18403310,285.000000,4,10989,360.0,150.0,4,166.0,2,205.0,...,4,5,24.0,390,2,2,2,2,2,2184
4,18403317,67.937500,4,18018,452.0,150.0,2,117.0,2,264.0,...,4,5,24.0,390,2,2,2,2,2,2184


## load model

- if model is no longer in memory (e.g. due to restarting of notebook)

In [18]:
classifiers = [
    joblib.load(model_dir_path + 'lgbm_' + str(index + offset) + '.joblib')
    for index in range(num_base_classifiers)
]

## do inference and get output

In [19]:
%%time

predictions = [
    classifier.predict(dataframe)
    for classifier in classifiers
]

prediction = predictions.pop()
for pred in predictions:
    prediction += pred

prediction /= 10  # equal weightage given to each base classifier - 0.1
del predictions

del classifiers
del dataframe

output_dataframe = pd.DataFrame({
    'TransactionID': transaction_id_data,
    'isFraud': pd.Series(prediction),
})

output_dataframe.to_csv(output_dir_path + 'avg_vote_lgbm_pred.csv', index=False)
output_dataframe.head()

CPU times: user 4h 55min 9s, sys: 22.9 s, total: 4h 55min 32s
Wall time: 27min 29s


,TransactionID,isFraud
0,3663549,0.000393
1,3663550,0.000112
2,3663551,0.000538
3,3663552,0.000407
4,3663553,0.000139


## Kaggle result

- public score: 0.941287
- public ranking: 3174 out of 6381 (~49.74%)
- private score: 0.913408
- private ranking: 2948 out of 6381 (~46.19%)